# 🎬 StreamSage: TMDb Data Exploration (Simplified)

**Goal**: Explore the TMDb 5000 Movies dataset - our single source of truth.

**What we'll build with this data**:
1. **Oracle (RAG)**: Answer questions about movies using plot summaries
2. **Content Recommender**: Find similar movies based on plot, genre, cast

**No merging, no complexity - just one clean dataset!**

In [ ]:
# Install required libraries
!pip install pandas numpy matplotlib seaborn kaggle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from collections import Counter

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

print("✅ Libraries loaded!")

## 📥 Download TMDb 5000 Dataset

**Note**: You need a Kaggle API key.
1. Go to https://www.kaggle.com/settings/account
2. Click "Create New API Token"
3. Upload the `kaggle.json` file when prompted below

In [ ]:
# Upload Kaggle API key
from google.colab import files
print("📤 Please upload your kaggle.json file:")
uploaded = files.upload()

# Setup Kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("✅ Kaggle configured!")

In [ ]:
# Download TMDb dataset
!kaggle datasets download -d tmdb/tmdb-movie-metadata
!unzip -q tmdb-movie-metadata.zip

print("✅ TMDb dataset downloaded!")

## 📊 Load and Explore Data

In [ ]:
# Load datasets
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

print(f"Movies shape: {movies.shape}")
print(f"Credits shape: {credits.shape}")

print("\n--- Column Names ---")
print("Movies:", movies.columns.tolist())
print("\nCredits:", credits.columns.tolist())

In [ ]:
# Display sample data
print("--- Sample Movie ---")
sample = movies.iloc[0]
print(f"Title: {sample['title']}")
print(f"\nOverview: {sample['overview']}")
print(f"\nGenres: {sample['genres']}")
print(f"\nKeywords: {sample['keywords']}")
print(f"\nRelease Date: {sample['release_date']}")
print(f"\nPopularity: {sample['popularity']}")

## 🧹 Data Quality Check

In [ ]:
# Check for missing values
print("📊 Missing Values Analysis")
missing = movies.isnull().sum()
missing_pct = (missing / len(movies) * 100).round(2)
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_pct
}).sort_values('Missing Count', ascending=False)

print(missing_df[missing_df['Missing Count'] > 0])

# Key insight: How many movies have plot summaries?
print(f"\n✅ Movies with overview: {movies['overview'].notna().sum()} / {len(movies)}")
print(f"✅ Coverage: {(movies['overview'].notna().sum() / len(movies) * 100):.1f}%")

## 🔧 Parse JSON Columns

TMDb stores genres, keywords, cast, and crew as JSON strings. We need to parse them.

In [ ]:
def parse_json_list(df, column):
    """Parse JSON column and extract 'name' field"""
    def extract_names(x):
        if pd.isna(x):
            return []
        try:
            data = json.loads(x)
            return [item['name'] for item in data]
        except:
            return []
    
    return df[column].apply(extract_names)

# Parse genres and keywords
movies['genres_list'] = parse_json_list(movies, 'genres')
movies['keywords_list'] = parse_json_list(movies, 'keywords')

print("✅ Parsed genres and keywords")

# Show example
print("\n--- Parsed Example ---")
print(f"Title: {movies.iloc[0]['title']}")
print(f"Genres: {movies.iloc[0]['genres_list']}")
print(f"Keywords: {movies.iloc[0]['keywords_list'][:5]}...")  # First 5

In [ ]:
# Parse cast and crew from credits
def get_director(crew_json):
    """Extract director from crew JSON"""
    if pd.isna(crew_json):
        return None
    try:
        crew = json.loads(crew_json)
        for person in crew:
            if person['job'] == 'Director':
                return person['name']
    except:
        return None
    return None

def get_top_cast(cast_json, n=5):
    """Extract top N cast members"""
    if pd.isna(cast_json):
        return []
    try:
        cast = json.loads(cast_json)
        return [person['name'] for person in cast[:n]]
    except:
        return []

# Merge credits with movies
movies = movies.merge(credits[['movie_id', 'cast', 'crew']], left_on='id', right_on='movie_id', how='left')

# Extract director and cast
movies['director'] = movies['crew'].apply(get_director)
movies['cast_list'] = movies['cast'].apply(get_top_cast)

print("✅ Extracted director and cast")

# Show example
print("\n--- Cast & Crew Example ---")
print(f"Title: {movies.iloc[0]['title']}")
print(f"Director: {movies.iloc[0]['director']}")
print(f"Cast: {movies.iloc[0]['cast_list']}")

## 📈 Data Visualization

In [ ]:
# Genre distribution
all_genres = [genre for genres in movies['genres_list'] for genre in genres]
genre_counts = Counter(all_genres)

plt.figure(figsize=(12, 6))
top_genres = dict(genre_counts.most_common(15))
plt.barh(list(top_genres.keys()), list(top_genres.values()), color='skyblue')
plt.xlabel('Number of Movies')
plt.title('Top 15 Genres in TMDb Dataset')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print(f"Total unique genres: {len(genre_counts)}")

In [ ]:
# Overview length distribution
movies['overview_length'] = movies['overview'].str.len()

plt.figure(figsize=(10, 5))
plt.hist(movies['overview_length'].dropna(), bins=50, color='coral', edgecolor='black')
plt.xlabel('Overview Length (characters)')
plt.ylabel('Number of Movies')
plt.title('Distribution of Plot Summary Lengths')
plt.axvline(movies['overview_length'].median(), color='red', linestyle='--', label=f'Median: {movies["overview_length"].median():.0f}')
plt.legend()
plt.show()

print(f"Average overview length: {movies['overview_length'].mean():.0f} characters")
print(f"Median overview length: {movies['overview_length'].median():.0f} characters")

In [ ]:
# Release year distribution
movies['year'] = pd.to_datetime(movies['release_date'], errors='coerce').dt.year

plt.figure(figsize=(12, 5))
movies['year'].hist(bins=50, color='purple', alpha=0.7, edgecolor='black')
plt.xlabel('Year')
plt.ylabel('Number of Movies')
plt.title('Movies by Release Year')
plt.show()

print(f"Year range: {movies['year'].min():.0f} - {movies['year'].max():.0f}")

## 🎯 Create Final Clean Dataset

In [ ]:
# Select and rename columns for clarity
clean_movies = movies[[
    'id',
    'title',
    'overview',
    'genres_list',
    'keywords_list',
    'cast_list',
    'director',
    'year',
    'popularity',
    'vote_average',
    'vote_count'
]].copy()

# Remove movies without overview (can't use for RAG)
clean_movies = clean_movies[clean_movies['overview'].notna()]

print(f"✅ Clean dataset created: {len(clean_movies)} movies")
print("\n--- Sample Clean Data ---")
display(clean_movies.head())

In [ ]:
# Create searchable text for each movie (for RAG and recommendations)
def create_movie_text(row):
    """Combine all text features into one searchable string"""
    parts = [
        f"Title: {row['title']}",
        f"Overview: {row['overview']}",
        f"Genres: {', '.join(row['genres_list'])}",
        f"Keywords: {', '.join(row['keywords_list'])}",
        f"Cast: {', '.join(row['cast_list'])}",
        f"Director: {row['director']}" if pd.notna(row['director']) else ""
    ]
    return " ".join([p for p in parts if p])

clean_movies['searchable_text'] = clean_movies.apply(create_movie_text, axis=1)

print("✅ Created searchable text")
print("\n--- Sample Searchable Text ---")
print(clean_movies.iloc[0]['searchable_text'][:500] + "...")

## 💾 Save Processed Data

In [ ]:
# Save to CSV
clean_movies.to_csv('tmdb_clean.csv', index=False)
print("✅ Saved to tmdb_clean.csv")

# Download file
try:
    from google.colab import files
    files.download('tmdb_clean.csv')
    print("📥 File downloaded!")
except:
    print("Not in Colab - file saved locally")

## 📋 Summary

### What We Have:
- ✅ **4,800+ movies** with complete metadata
- ✅ **Plot summaries** for Oracle Q&A
- ✅ **Genres, keywords, cast** for content-based recommendations
- ✅ **Searchable text** ready for vector embedding

### Next Steps:
1. **Notebook 2**: Build content-based recommender (TF-IDF + Cosine Similarity)
2. **Notebook 3**: Create vector embeddings for Oracle RAG
3. **Integration**: Load into Docker services

**No merging, no complexity - just clean, usable data!** 🎉